In [1]:
from IDRiDClassifier import IDRiDClassifier
from IDRiDGradingDataset import IDRiDGradingDataset
import copy
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy
import numpy as np
import torch
import pandas as pd
from classifier_utils import train, test

In [3]:
# Prepare data
root_dir_train = '/Users/khalilouardini/Desktop/projects/dlmi/B. Disease Grading/1. Original Images/a. Training Set'
csv_file_train = '/Users/khalilouardini/Desktop/projects/dlmi/B. Disease Grading/2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv'

root_dir_test = '/Users/khalilouardini/Desktop/projects/dlmi/B. Disease Grading/1. Original Images/b. Testing Set'
csv_file_test = '/Users/khalilouardini/Desktop/projects/dlmi/B. Disease Grading/2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv'
bengraham = False

# transform (preprocessing)
scale = transforms.Resize((64, 64))
to_tensor = transforms.ToTensor()
composed = transforms.Compose([scale,
                               to_tensor])
# train and test set
train_set = IDRiDGradingDataset(csv_file_train, root_dir_train, composed, bengraham)
test_set = IDRiDGradingDataset(csv_file_test, root_dir_test, composed, bengraham)

#dataloaders
batch_size = 4
train_loader = DataLoader(train_set, batch_size=batch_size,
                            shuffle=True, num_workers=0)
test_loader = DataLoader(test_set, batch_size=batch_size,
                            shuffle=True, num_workers=0)

print(f"# batches in train set {len(train_loader)} | # batches in test set {len(test_loader)}")

# batches in train set 104 | # batches in test set 26


In [ ]:
# model
use_cuda = torch.cuda.is_available()
model = IDRiDClassifier()
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    model = torch.nn.DataParallel(model)
model.to(device)

# intialize variables
n_epochs = 1
lr = 1e-4
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
best_model = train(model, train_loader, criterion, optimizer, n_epochs, device)
test(model, test_loader, criterion, device)
print("done")